In [121]:
import torch

In [122]:
torch.cuda.is_available()

True

In [123]:
# Get device ID
torch.cuda.current_device()

0

In [124]:
#  Get device name
torch.cuda.get_device_name()

'GeForce RTX 2060'

In [125]:
# Get memory allocate
torch.cuda.memory_allocated()

64000

In [126]:
# Get memory cached
torch.cuda.memory_reserved()

2097152

In [127]:
a = torch.FloatTensor([1.0, 2.0])

In [128]:
a

tensor([1., 2.])

In [129]:
a.device

device(type='cpu')

In [130]:
a = torch.FloatTensor([1.0,2.0]).cuda()

In [131]:
a.device

device(type='cuda', index=0)

In [132]:
torch.cuda.memory_allocated()

64000

# Send model to GPU

In [133]:
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [134]:
class Model(nn.Module):
    def __init__(self, in_features=4, h1=8, h2=9, out_features=3):
        super().__init__()
        self.fc1 = nn.Linear(in_features, h1)   # input layer
        self.fc2 = nn.Linear(h1, h2)            # hiden layer
        self.out = nn.Linear(h2, out_features)  # output layer

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.out(x)
        return x

In [135]:
torch.manual_seed(42)
model = Model()

In [136]:
next(model.parameters()).is_cuda

False

In [137]:
gpumodel = model.cuda()

In [138]:
next(gpumodel.parameters()).is_cuda

True

In [139]:
df = pd.read_csv('../datasets/iris.csv')

In [140]:
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0.0
1,4.9,3.0,1.4,0.2,0.0
2,4.7,3.2,1.3,0.2,0.0
3,4.6,3.1,1.5,0.2,0.0
4,5.0,3.6,1.4,0.2,0.0


In [141]:
X = df.drop('target', axis=1).values

In [142]:
y = df['target'].values

In [143]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=99)

In [144]:
X_train = torch.FloatTensor(X_train).cuda()
X_test = torch.FloatTensor(X_test).cuda()

y_train = torch.LongTensor(y_train).cuda()
y_test = torch.LongTensor(y_test).cuda()

In [145]:
trainloader = DataLoader(X_train, batch_size=60, shuffle=True, pin_memory=True)
testloader = DataLoader(X_test, batch_size=60, shuffle=False, pin_memory=True)

In [146]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(gpumodel.parameters(), lr=0.01)

In [147]:
import time
start = time.time()

epochs = 100
losses = []

for i in range(epochs):
    y_pred = gpumodel.forward(X_train)
    loss = criterion(y_pred, y_train)
    losses.append(loss)

    if i%10 == 0:
        print(f'Epoch: {i} loss: {loss.item()}')

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

end = time.time() - start
print(f"TOTAL TIME: {end}")

Epoch: 0 loss: 1.0961973667144775
Epoch: 10 loss: 0.9154850244522095
Epoch: 20 loss: 0.7587660551071167
Epoch: 30 loss: 0.6157783269882202
Epoch: 40 loss: 0.45067501068115234
Epoch: 50 loss: 0.28452008962631226
Epoch: 60 loss: 0.1734197437763214
Epoch: 70 loss: 0.11226236820220947
Epoch: 80 loss: 0.08438906818628311
Epoch: 90 loss: 0.0715310275554657
TOTAL TIME: 0.11789584159851074


In [148]:
correct = 0
with torch.no_grad():
    for i, data in enumerate(X_test):
        y_val = gpumodel.forward(data)
        print(f'{i+1:2}. {str(y_val):38} {y_test[i]}')
        if y_val.argmax().item() == y_test[i]:
            correct += 1

print(f'\n{correct} out of {len(y_test)} = {100*correct/len(y_test):.2f}% correct')

 1. tensor([-5.5697,  4.2902,  7.0894], device='cuda:0') 2
 2. tensor([ 6.8973,  1.9857, -5.5731], device='cuda:0') 0
 3. tensor([0.3491, 7.2320, 1.1525], device='cuda:0') 1
 4. tensor([-7.2287,  3.3358,  8.7401], device='cuda:0') 2
 5. tensor([ 7.6895,  2.2993, -6.2576], device='cuda:0') 0
 6. tensor([-0.7448,  5.7466,  2.1155], device='cuda:0') 1
 7. tensor([-1.6959,  5.7579,  3.1351], device='cuda:0') 1
 8. tensor([-7.4970,  3.1459,  8.9806], device='cuda:0') 2
 9. tensor([ 6.9394,  2.5088, -5.5405], device='cuda:0') 0
10. tensor([-3.4428,  4.6571,  4.8652], device='cuda:0') 1
11. tensor([-9.3879,  3.1420, 10.9958], device='cuda:0') 2
12. tensor([-1.4706,  6.0932,  2.9288], device='cuda:0') 1
13. tensor([-3.9702,  4.5112,  5.4068], device='cuda:0') 2
14. tensor([-0.8603,  5.8681,  2.2535], device='cuda:0') 1
15. tensor([-6.4924,  3.7144,  7.9976], device='cuda:0') 2
16. tensor([ 8.0639,  2.4475, -6.5811], device='cuda:0') 0
17. tensor([ 7.4667,  2.2111, -6.0651], device='cuda:0') 0
